In [1]:
import json
import numpy as np

from sklearn.model_selection import ParameterSampler
from scipy.stats import randint, uniform

In [2]:
parameter_grid = {
#     'epochs': [25, 50, 100, 200], # it doesn't make sense to search epochs, when we can use early stopping
    'batch_size': [64, 128, 256, 512, 1024],
    'learning_rate': [0.0001, 0.001, 0.01], # TODO: can this be optimized with a learning rate scheduler?
    'dropout_rate': [0.0, 0.05, 0.1, 0.15, 0.2], # TODO: can we sample from a continuout distribution?
    'latent_dim': [256, 512, 768, 1024],
    'embedding_dim': [32, 50, 64, 96, 128],
    'vocabulary_size': [2000, 3000, 4000, 5000, 6000, 7000],
    'max_input_seq_length': [64, 96, 128, 160, 192],
    'max_output_seq_length': [5, 6, 7, 8],
#     'learning_rate_distro': lognorm([0.01], loc=-0.8),
#     'bi_lstm': [True, False], # not supported yet
}

In [3]:
experiments = {
    f'experiment_{i}': params
    for i, params in enumerate(ParameterSampler(
        parameter_grid,
        n_iter=20,
        random_state=0,
    ))
}

In [4]:
print(json.dumps(experiments, indent=2))

{
  "experiment_0": {
    "vocabulary_size": 3000,
    "max_output_seq_length": 6,
    "max_input_seq_length": 64,
    "learning_rate": 0.0001,
    "latent_dim": 512,
    "embedding_dim": 32,
    "dropout_rate": 0.05,
    "batch_size": 128
  },
  "experiment_1": {
    "vocabulary_size": 5000,
    "max_output_seq_length": 8,
    "max_input_seq_length": 96,
    "learning_rate": 0.001,
    "latent_dim": 768,
    "embedding_dim": 32,
    "dropout_rate": 0.2,
    "batch_size": 1024
  },
  "experiment_2": {
    "vocabulary_size": 6000,
    "max_output_seq_length": 7,
    "max_input_seq_length": 192,
    "learning_rate": 0.001,
    "latent_dim": 1024,
    "embedding_dim": 50,
    "dropout_rate": 0.05,
    "batch_size": 512
  },
  "experiment_3": {
    "vocabulary_size": 7000,
    "max_output_seq_length": 6,
    "max_input_seq_length": 160,
    "learning_rate": 0.001,
    "latent_dim": 1024,
    "embedding_dim": 64,
    "dropout_rate": 0.2,
    "batch_size": 1024
  },
  "experiment_4": {
    "